In [ ]:
!pip install ultralytics ttach roboflow

In [ ]:
import numpy as np
import pandas as pd
import regex as re
import os
import shutil

from roboflow import Roboflow
from PIL import Image

from ultralytics import YOLO

import yaml
from zipfile import ZipFile

### Data Setup

In [ ]:
rf = Roboflow(api_key="6kQe6c01IeFKXTyChZff")
project = rf.workspace("aisummerschool2024").project("tka-localization-fmeno")
version = project.version(1)
dataset = version.download("yolov9", location="/content/datasets/TKA-Localization")


In [ ]:
dataset.location

### Few-Shot YOLOv8 Training

In [ ]:
# Import YOLO from Ultralytics library
from ultralytics import YOLO
import os

In [ ]:
model = YOLO("yolov9c.pt")  # load a pretrained model (recommended for training)

In [ ]:
dataset.name

In [ ]:
# Train on few-shot dataset
# Verify the correct path to your data.yaml file
data_yaml_path = f'{dataset.location}/data.yaml'
print(f"Checking for data.yaml at: {data_yaml_path}")

if os.path.exists(data_yaml_path):
    print("data.yaml found. Proceeding with training.")
    # Train on few-shot dataset
    metrics = model.train(data=data_yaml_path, epochs=100)
else:
    print("data.yaml not found. Please check the path and ensure the file exists.")

In [ ]:
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
# Evaluate on test set
results = model.val(split="test")

In [ ]:
results

In [ ]:
# Single image prediction
result = model.predict("/content/datasets/TKA-Localization/test/images/00940404_jpg.rf.952202247cb2c0871f612ca98c8b2c2a.jpg")


In [ ]:
# Multiple image prediction
trained_model_pth = "runs/detect/train/weights/best.pt"
model = YOLO(trained_model_pth)

test_root = f"/content/datasets/TKA-Localization/test/images"
test_imgs = [os.path.join(test_root, img) for img in os.listdir(test_root)]
result = model.predict(test_imgs, save=True)

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    Each box is defined as a list of 4 elements [x1, y1, x2, y2]
    where (x1, y1) is the top-left corner, and (x2, y2) is the bottom-right corner.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculate the area of intersection rectangle
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate the area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area.
    iou = inter_area / float(box1_area + box2_area - inter_area)

    return iou

In [ ]:
def convert_ground_truth_to_coco(x, y, w, h, img_width=600, img_height=600):
    # Convert normalized coordinates to pixel coordinates
    x *= img_width
    y *= img_height
    w *= img_width
    h *= img_height

    # Calculate top-left and bottom-right corners
    x1 = x - w /2
    y1 = y - h /2
    x2 = x + w /2
    y2 = y + h /2

    return [x1, y1, x2, y2]

In [ ]:
def get_ground_truth_box(image_path):
    with open(image_path, 'r') as file:
        ground_truth_boxes = []
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:
                _, x_center, y_center, width, height = map(float, parts)
                ground_truth_boxes.append([x_center, y_center, width, height])
    return ground_truth_boxes

In [ ]:
for img in test_imgs:
    label_pth = img.replace("images", "labels")
    label_pth = label_pth.replace(".jpg", ".txt")
    ground_truth_boxes = get_ground_truth_box(label_pth)

    results = model.predict(img)
    bboxes = results[0].boxes.xyxy.cpu().numpy()
    for pred_box in bboxes:
        best_iou = 0
        best_gt_box = None
        for gt_box in ground_truth_boxes:
            conversion = convert_ground_truth_to_coco(gt_box[0], gt_box[1], gt_box[2], gt_box[3])
            iou = calculate_iou(pred_box, conversion)
            if iou > best_iou:
                best_iou = iou
                best_gt_box = conversion

        print(f"Image {img}, Predicted box: {pred_box}, Best matching ground truth box: {best_gt_box}, IoU: {best_iou}")

